In [2]:
from copyreg import pickle
!python -V

Python 3.10.4


MLflow setup:

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root ./mlruns --host 0.0.0.0 --port 5000`

To start MLflow Tracking Server enabled with proxied artifact storage access:
( see more in : [https://www.mlflow.org/docs/latest/tracking.html#where-runs-are-recorded](https://www.mlflow.org/docs/latest/tracking.html#where-runs-are-recorded) )

In [3]:
S3_BUCKET = 's3://mlflow-enkidupal-experiments'
f'mlflow server --host 0.0.0.0 --port 5000 --serve-artifacts --artifacts-destination {S3_BUCKET}/'


'mlflow server --host 0.0.0.0 --port 5000 --serve-artifacts --artifacts-destination s3://mlflow-enkidupal-experiments/'

To rerun the server, this command has to be run in *correct* directory,
where previous mlflow server was run, to point backend-store-uri to mlflow.db !

`cd project_root`


 `mlflow server --host 0.0.0.0 --port 5000 --serve-artifacts --artifacts-destination s3://mlflow-enkidupal-experiments/`

Import libraries

In [4]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import mlflow

import pandas as pd
import numpy as np
from toolz import compose
import pickle

from sklearn.model_selection import train_test_split

from pandas.core.common import SettingWithCopyWarning
from matplotlib_inline import backend_inline

import warnings

backend_inline.set_matplotlib_formats('svg')
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [5]:
import os

_data_root = os.path.join("./../", 'data')
_data_root_raw = os.path.join(_data_root, 'raw')
_data_root_processed =  os.path.join(_data_root, 'processed')

_train_dirpath = os.path.join(_data_root_raw, "train")
_train_filepath = os.path.join(_train_dirpath, "train.csv")
_test_dirpath = os.path.join(_data_root_raw, "test")
_test_filepath = os.path.join(_test_dirpath, "test.csv")

_train_processed_dirpath = os.path.join(_data_root_processed, "train")
_train_processed_filepath = os.path.join(_data_root_processed, "train.csv")
_valid_processed_dirpath = os.path.join(_data_root_processed, "valid")
_valid_processed_filepath = os.path.join(_data_root_processed, "valid.csv")
_test_processed_dirpath = os.path.join(_data_root_processed, "test")
_test_processed_filepath = os.path.join(_data_root_processed, "test.csv")

os.makedirs(_train_dirpath, exist_ok=True)
os.makedirs(_test_dirpath, exist_ok=True)
os.makedirs(_train_processed_dirpath, exist_ok=True)
os.makedirs(_valid_processed_dirpath, exist_ok=True)
os.makedirs(_test_processed_dirpath, exist_ok=True)


## Download data from kaggle, unzip it and copy it to data folder


In [6]:
def download_data():
    !kaggle competitions download -c titanic -p {_data_root} --force
    !unzip -o {_data_root}/"titanic.zip" -d {_data_root}
    !cp {_data_root}/"train.csv" {_train_filepath}
    !cp {_data_root}/"test.csv" {_test_filepath}

    # clean up
    !rm  {_data_root}/*.csv  {_data_root}/*.zip

def extract_target(data: pd.DataFrame, target="Survived"):
    targets = data[target].values
    return targets

def preprocess_df_2(df: pd.DataFrame, transforms, categorical, numerical):
    """Return processed features dict and target."""

    # Apply in-between transformations
    df = compose(*transforms[::-1])(df)

    # For dict vectorizer: int = ignored, str = one-hot
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess_df(df: pd.DataFrame, transforms, categorical, numerical):
    """Return processed features dict and target."""

    # Apply in-between transformations
    df = compose(*transforms[::-1])(df)

    # For dict vectorizer: int = ignored, str = one-hot
    df[categorical] = df[categorical].astype(str)

    # Convert dataframe to feature dictionaries
    feature_dicts = df[categorical + numerical].to_dict(orient='records')

    return feature_dicts


def read_data(filename):
    """Return processed features dict and target."""

    # Load dataset
    if filename.endswith('parquet'):
        df = pd.read_parquet(filename)
    elif filename.endswith('csv'):
        df = pd.read_csv(filename)
    else:
        raise "Error: not supported file format."

    return df


def preprocess_no_extract_target(filename, transforms, categorical, numerical):
    df = read_data(filename)

    feature_dicts = preprocess_df(df, transforms, categorical, numerical)

    return feature_dicts

def split_train_read(filename: str, val_size=0.2, random_state=42):
    df_train_full = read_data(filename)

    df_train, df_val = train_test_split(df_train_full, test_size=val_size, random_state=random_state)
    return df_train, df_val

def save_preprocessed(df: pd.DataFrame, path):
    df.to_csv(path)

def dump_pickle(obj, filename):
    with open(filename, "wb") as f_out:
        return pickle.dump(obj, f_out)

def run():
    mlflow.set_tracking_uri("http://0.0.0.0:5000")
    mlflow.set_experiment("titanic-experiment")
    mlflow.sklearn.autolog()

    with mlflow.start_run():
        download_data()
        transforms = []
        target = 'Survived'
        categorical = ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
        numerical = ['Fare']

        df_train, df_val = split_train_read(_train_filepath, val_size=0.2, random_state=42)

        train_dicts, y_train = preprocess_df(df_train, transforms, categorical, numerical), extract_target(df_train)
        val_dicts, y_val = preprocess_df(df_val, transforms, categorical, numerical), extract_target(df_val)

        df_test = read_data(_test_filepath)
        test_dicts = preprocess_df(df_test, transforms, categorical, numerical)

        # Fit all possible categories
        dv = DictVectorizer()
        dv.fit(train_dicts)

        X_train = dv.transform(train_dicts)
        X_val = dv.transform(val_dicts)
        X_test = dv.transform(test_dicts)

        model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

        model.fit(X_train, y_train)

        with open("../models/preprocessor.b", "wb") as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

        #    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
        #mlflow.log_artifact(local_path=os.path.join("../models", "rfc.pkl"), artifact_path="models_pickle")
        mlflow.sklearn.log_model(model, artifact_path="models_pickle")

        y_pred = model.predict(X_val)

        accuracy = np.round(accuracy_score(y_val, y_pred), 4)
        mlflow.log_metric("accuracy", accuracy)

        print(accuracy)


In [7]:
transforms = []
target = 'Survived'
categorical = ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
numerical = ['Fare']

df_train, df_val = split_train_read(_train_filepath, val_size=0.2, random_state=42)
df_train_preprocessed = preprocess_df_2(df_train, transforms, categorical, numerical)

df_train_preprocessed = df_train_preprocessed.reset_index()
#df_train_preprocessed[categorical].head()

#df_train_preprocessed[categorical].isna()

In [26]:
df_train_preprocessed.head()

df_train_preprocessed[categorical][0:1].to_dict()

{'Sex': {0: 'male'},
 'Pclass': {0: '3'},
 'Embarked': {0: 'Q'},
 'SibSp': {0: '0'},
 'Parch': {0: '0'}}

In [27]:
run()

  0%|                                               | 0.00/34.1k [00:00<?, ?B/s]
100%|██████████████████████████████████████| 34.1k/34.1k [00:00<00:00, 1.82MB/s]
Archive:  ./../data/titanic.zip
  inflating: ./../data/gender_submission.csv  
  inflating: ./../data/test.csv      
  inflating: ./../data/train.csv     
0.8045


In [ ]:
import boto3
import pickle



In [51]:
import boto3
import pickle

import src.features.build_features

RUN_ID = 'd6dc868dda9e41ecaf536a91f41e059e'

logged_model = f's3://mlflow-enkidupal-experiments/1/{RUN_ID}/artifacts/models_pickle/'

model = mlflow.pyfunc.load_model(logged_model)

s3client = boto3.client('s3' )

response = s3client.get_object(Bucket='mlflow-enkidupal-experiments', Key=f'1/{RUN_ID}/artifacts/preprocessor/preprocessor.b')

body = response['Body'].read()
preprocessor = pickle.loads(body)

print(model)
print(preprocessor)

mlflow.pyfunc.loaded_model:
  artifact_path: models_pickle
  flavor: mlflow.xgboost
  run_id: d6dc868dda9e41ecaf536a91f41e059e

Pipeline(steps=[('dictvectorizer', DictVectorizer())])


In [37]:
from src.features.build_features import preprocess_test

passenger_X = {
    "Sex": ['female'],
    "Pclass": [0],
    "Embarked": ['C'],
    "SibSp": [1],
    "Parch": [10],
    "Fare": [69.34]
}

df_X = pd.DataFrame.from_dict(passenger_X)


prep = preprocess_test(df_X, preprocessor )
print('prep: ', prep)

prediction = model.predict(prep)
prediction[0]



prep:    (0, 4)	69.34
  (0, 5)	10.0
  (0, 6)	0.0
  (0, 7)	1.0
  (0, 9)	1.0


0.8314826

In [68]:
from sklearn.metrics import accuracy_score

df_train_full = read_data(_train_filepath)
df_train_full.head()


targets = extract_target(df_train_full)
#targets = list(targets)
print(type(targets), targets.dtype)
prep = preprocess_test(df_train_full, preprocessor)

y_pred = model.predict(prep)
predictions = np.array([round(value) for value in y_pred])
print(type(predictions), predictions.dtype)
acc = accuracy_score(targets, y_pred.round())
acc

<class 'numpy.ndarray'> int64
<class 'numpy.ndarray'> int64


0.8328651685393258

In [62]:


two_passengers = [
    {
        "Sex": ['female'],
        "Pclass": [0],
        "Embarked": ['C'],
        "SibSp": [1],
        "Parch": [10],
        "Fare": [69.34]
    },
    {
        "Sex": ['male'],
        "Pclass": [1],
        "Embarked": ['C'],
        "SibSp": [0],
        "Parch": [2],
        "Fare": [8]
    }
]

df = pd.json_normalize(two_passengers, record_path='location', meta=['id', 'name'])

False

In [69]:
categorical = ['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch']
numerical = ['Fare']

all_columns = categorical + numerical
all_columns

['Sex', 'Pclass', 'Embarked', 'SibSp', 'Parch', 'Fare']

In [61]:
predictions

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,


In [58]:
type(targets), type(predictions)

(numpy.ndarray, list)